In [ ]:
############################################################################
#######        FUTURES MODEL - Including margin requirements         #######
############################################################################
#######                          Description                         #######
############################################################################
####### This model aims to simulate the returns from investing in    #######
####### a given futures contract, as well as measuring the margin    #######
####### requirements. After the simulation you can inspect the       #######
####### recorded variables to see if your margin account would       #######
####### be able to handle each margin call during the drawdowns.     #######
####### If your margin account is not able to meet the margin        #######
####### calls the cash_account will go to 0, in reality your         #######
####### position would have been closed by the broker. It will       #######
####### come a message in the log as well, however it might be       #######
####### hard to find. You can experiment with contracts bought       #######
####### to see how many postitions you could have open before        #######
####### not being able to meet the margin calls.                     #######
############################################################################
#######                          Parameters                          #######
############################################################################
####### Choose your capital base and how much to have as cash        #######
####### reserve(in percentage). If you set 0% all of the capital     #######
####### will be allocated to the margin_account. Find the normal     #######
####### margin requirements for the contract to trade(per contract). #######
####### select the contract to trade, how many contracts, and        #######
####### how many days before the closing day of the contract         #######
####### to roll over (standard is two). The model engourage to       #######
####### experiment with how many contracts you can sustain.          #######
############################################################################

import pandas as pd
from datetime import timedelta
from zipline.utils.calendars import get_calendar

def initialize(context):
    ##################################################################
    ##                Traders account parameters                    ##
    ##################################################################
    #Capital allocation
    context.available_funds = 100000
    context.cash_reserve = 0.2
    #Margins per contract
    context.initial_margin_requirement = 4900
    context.maintenance_margin = 4500
    ##################################################################
    ##                    Contract parameters                       ##
    ##################################################################
    #Select future to trade
    context.contract_to_trade = "GC"
    #How many contracts to buy
    context.contracts = 6
    #Select how many days before contract expiery to roll over
    context.roll_days = 2
    ##################################################################
    ##################################################################
    
    
    ##################################################################
    #                          MODEL                                 #
    ##################################################################
    
    context.future = continuous_future(context.contract_to_trade,
                                       offset=0, 
                                       roll='volume',
                                       adjustment='mul')
    
    
    schedule_function(func=contract_data,
                      date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open())
 
    schedule_function(func=buy_logic,
                      date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open())
    
    schedule_function(func=roll_contract,
                      date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open())
    
    schedule_function(func=get_price,
                      date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open())
    
    schedule_function(func=margin_calc,
                      date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open())
    
    schedule_function(func=record_vars, 
                      date_rule=date_rules.every_day(), 
                      time_rule=time_rules.market_close())
    
    
    #Set up boolean for functions that only run once
    context.buy_logic_bol = 0
    context.starting_price = 0
    #Set up boolean for functions that run after initial trading day
    context.margin_calc = 0
    
    # We are going to use this calendar to compute how many trading days
    # away from the auto_close_date of a contract we are.
    context.futures_calendar = get_calendar('us_futures')
    
    #Build the margain account
    context.margin_account = context.available_funds*(1-context.cash_reserve)
    #Build cash account
    context.cash_account = context.available_funds * context.cash_reserve
    #Build the margin limit
    context.margin_minimum = context.contracts*context.maintenance_margin
    
    #first day price
    context.price = 0
    
def margin_calc(context, data):
    if context.margin_calc >= 1:
        change = context.new_price-context.price
        multiplier = context.contract.multiplier
        pct_change = (change/context.price)*100
        #Calculate the new margin balance
        context.margin_account = context.margin_account + (change*
                                                           multiplier*
                                                           context.contracts)
        #Calculate the minimum required margin
        context.margin_minimum = context.contracts*context.maintenance_margin
        
        #Assaign the the new price as previous day price for next day calc. 
        context.price = context.new_price
        print("MARGIN ACCOUNT BALANCE: {}$".format(round(context.margin_account)))
        print("Daily change in price from last trading day {}%".format(round(pct_change,2)))
        
        if context.margin_account<context.margin_minimum:
            cash_need = context.margin_minimum-context.margin_account
            print("WARNING: DRAWDOWN EXCEEDS MINIMUM REQUIRED MARGIN BALANCE")
            if cash_need>context.cash_account:
                print("!!SIMULATION FAILED!!: Account would have been closed. Not enough funds")
                context.cash_account = 0
            else:
                context.margin_account=context.margin_account+cash_need
                context.cash_account = context.cash_account-cash_need
                print("Cash account has been deducted {}$, new  balance {}".format(cash_need,context.cash_account))
        
    context.margin_calc = context.margin_calc+1
        
    
def get_price(context, data):
    if context.starting_price == 0:
        context.price = data.current(context.future,"price")
        context.starting_price = context.starting_price+1
    
    
def contract_data(context, data):
    context.contract = data.current(context.future,"contract")
    context.new_price = data.current(context.future,"price")
    

def buy_logic(context, data):
    if context.buy_logic_bol == 0:
        order_target(context.contract,context.contracts)
        print("Initial contract {} bought".format(context.contract))
        context.buy_logic_bol = context.buy_logic_bol+1

def roll_contract(context, data):
    todays_date = get_datetime('US/Eastern')
    my_positions=context.portfolio.positions
    # Get the forward looking chain of contracts for our continuous future.
    contract_chain = data.current_chain(context.future)
    
    if len(my_positions)>0:
        for position in context.portfolio.positions:
            date_close = position.auto_close_date
            distance = context.futures_calendar.session_distance(todays_date,date_close)
            
            if distance <= context.roll_days:
                order_target(position,0)
                order_target(contract_chain[1],context.contracts)
                print("Position on {} is closed due to rollover. New position: {}, Number of contracts: {}".format(position,contract_chain[1],context.contracts))
                
def record_vars(context, data):
    # Plot the prices of MSFT and AAPL
    record(margin_account=context.margin_account,
           min_margin=context.margin_minimum,
           cash_account=context.cash_account
          )